This Notebook intends to compare our trained models and the ensemble learning technique, with huggingface's model available on huggingface. Article reference [here](https://www.sciencedirect.com/science/article/pii/S1544612324000096)

# Data Import

In [1]:
import pandas as pd
from datasets import load_dataset


In [2]:
social_data = load_dataset("ESGBERT/social_2k")['train'].to_pandas().rename(columns={'soc': 'esg_category'})
gov_data = load_dataset("ESGBERT/governance_2k")['train'].to_pandas().rename(columns={'gov': 'esg_category'})
env_data = load_dataset("ESGBERT/environmental_2k")['train'].to_pandas().rename(columns={'env': 'esg_category'})

In [14]:
social_data['esg_category'] = social_data['esg_category'].map({1: 'social', 0: 'non-esg'}).fillna(social_data['esg_category'])
gov_data['esg_category'] = gov_data['esg_category'].map({1: 'gouvernance', 0: 'non-esg'}).fillna(gov_data['esg_category'])
env_data['esg_category'] = env_data['esg_category'].map({1: 'environnemental', 0: 'non-esg'}).fillna(env_data['esg_category'])

labels = ['social', 'gouvernance', 'environnemental']

# sample 150 social, 150 governance, 150 environmental from each dataset and 50 non-esg from each dataset
test_data = pd.concat([social_data[social_data['esg_category'] == 'social'].sample(150,random_state=42),
                       gov_data[gov_data['esg_category'] == 'gouvernance'].sample(150,random_state=42),
                       env_data[env_data['esg_category'] == 'environnemental'].sample(150,random_state=42),
                       social_data[social_data['esg_category'] == 'non-esg'].sample(50,random_state=42),
                       gov_data[gov_data['esg_category'] == 'non-esg'].sample(50,random_state=42),
                       env_data[env_data['esg_category'] == 'non-esg'].sample(50,random_state=42)], ).drop(columns=['Unnamed: 0'])
test_data.reset_index(drop=True, inplace=True)
test_data.to_csv("ESGBert_test_data.csv", index=False)
test_data

['social' 'gouvernance' 'environnemental' 'non-esg']


,text,esg_category
0,We have a dedicated team who manage a portfoli...,social
1,A listed entity should disclose whether it has...,social
2,Our suppliers' activities support our overall ...,social
3,This year we raised the bar on our environment...,social
4,"the BGC’s AdTech forum, which includes a parti...",social
...,...,...
595,The information reported by the Company concer...,non-esg
596,One measure of loyalty is the employees’ years...,non-esg
597,The Board has established a remuneration commi...,non-esg
598,This development offers additional growth pote...,non-esg
